In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import sklearn
import statistics
import math 



In [3]:

data=pd.read_csv("/content/drive/My Drive/sc_lab/SPECTF_test.csv")
data=pd.DataFrame(data)
print(data)
d2=data["Class"]
data.drop(["Class"],axis=1,inplace=True)
d1=data
d1=np.array(d1)
d2=np.array(d2)


     Attr_1  Attr_2  Attr_3  Attr_4  ...  Attr_42  Attr_43  Attr_44  Class
0        32      41      76      34  ...        8       18       11    Yes
1        76      65      60      40  ...       30       24       21    Yes
2        60      51      75      60  ...       31       46       30    Yes
3        64      60      71      69  ...       64       61       56    Yes
4        65      69      66      76  ...       65       63       61    Yes
..      ...     ...     ...     ...  ...      ...      ...      ...    ...
105      70      75      72      72  ...       67       56       54     No
106      59      57      67      71  ...       63       56       56     No
107      67      64      73      75  ...       80       67       68     No
108      68      65      72      72  ...       56       58       44     No
109      66      54      69      66  ...       68       59       59     No

[110 rows x 45 columns]


In [4]:
class_to_no={}
class_to_no['Yes']=0
class_to_no['No']=1


In [5]:
no_to_class={}
no_to_class[0]='Yes'
no_to_class[1]='No'


In [6]:

def get_group(x,y):
  group=[[[] for i in range(44)] for i in range(2)]
  for i in range(len(x)):
    for j in range(44):
      group[class_to_no[y[i]]][j].append(x[i][j])
  return group



In [7]:

def get_group_mean(group):
  group_mean=[[] for i in range(2)]
  for i in range(len(group)):
    for j in range(len(group[i])):
      if(len(group[i][j])==0):
        group_mean[i].append(0)
      else:
        group_mean[i].append(statistics.mean(group[i][j]))

  return group_mean



In [8]:

def get_stddev(group):
  group_stdev=[[] for i in range(2)]
  for i in range(len(group)):
    for j in range(len(group[i])):
      if(len(group[i][j])<2):
        group_stdev[i].append(0)
      else:
        group_stdev[i].append(statistics.stdev(group[i][j]))
  
  return group_stdev



In [9]:
#P(x k |C i ) is usually computed based on Gaussian distribution
def prob(x,mean,stddev):
  if(stddev==0):
    return 0
  return (math.exp(-((x-mean)**2)/(2*(stddev**2))))/(math.sqrt(2*math.pi)*stddev)

In [10]:

def get_count_class(y):
  count_class={}
  count_class['Yes']=0
  count_class['No']=0
  for i in y:
    count_class[i]+=1
  return count_class

In [11]:

def get_prob_class(count_class,y):
  prob_class={}
  for i in count_class:
    prob_class[i]=count_class[i]/len(y)
  return prob_class

In [12]:
def predict(group,group_mean,group_stdev,prob_class,x_test):
  predict_class=[]
  for i in range(len(x_test)):

    temp=[]
    for j in range(len(group)):
      p=1
      for k in range(44):
        p=p*prob(x_test[i][k],group_mean[j][k],group_stdev[j][k])
      p=p*prob_class[no_to_class[j]]
      temp.append(p)
    
    y=0
    x=0
  
    for i in range(len(temp)):
      if(temp[i]>x):
        y=i
        x=temp[i]

    predict_class.append(y)
  return predict_class  
  


In [13]:
def classification_report(cm):
    # accuracy = 0
    # total = np.sum(cm)
    # diagonal = np.sum(np.diag(cm))
    # accuracy=(diagonal/total)*100
    # print("Accuracy : => ",(diagonal/total)*100)
    TP=cm[0][0]
    TN=cm[1][1]
    FN=cm[0][1]
    FP=cm[1][0]
    accuracy = ((TP+TN)/(FP+FN+TP+TN))*100
    print("Accuracy :=>",accuracy)
    return accuracy

In [14]:
print("SPECTF dataset")
Avg_accuracy=0
block=int(len(data)/10)
M=[[0 for i in range(2)] for i in range(2)]
for i in range(10):
  temp_x_data=[]
  temp_y_data=[]
  
  x_test=[]
  y_test=[]
  
  point=[]
  for j in range(block):
    point.append(block*i+j)

  
  for j in range(len(data)):
    if(j in point):
      x_test.append(d1[j])
      y_test.append(d2[j])
    else:
      temp_x_data.append(d1[j])
      temp_y_data.append(d2[j])
  # print(len(temp_x_data),len(temp_y_data))
  group=get_group(temp_x_data,temp_y_data)
  
  group_mean=get_group_mean(group)
  group_stdev=get_stddev(group)
  count_class=get_count_class(temp_y_data)
  prob_class=get_prob_class(count_class,temp_y_data)
  
  predict_class=predict(group,group_mean,group_stdev,prob_class,x_test)
  print("Fold ",i+1)
  # for j in range(len(predict_class)):
  #   print(no_to_class[predict_class[j]],y_test[j])
  
  matrix=[[0 for j in range(2)] for j in range(2)]

  for j in range(len(x_test)):
    matrix[class_to_no[y_test[j]]][predict_class[j]]+=1

  for p in range(2):
    for q in range(2):
      M[p][q]+=matrix[p][q]

  print("Confusion matrix")
  print(matrix)

  accu=classification_report(matrix)
  
  Avg_accuracy+=accu
  print()


print("Final confusion matrix")
print(M)

Avg_accuracy/=10
print()
print("Average accuracy=>",Avg_accuracy)

SPECTF dataset
Fold  1
Confusion matrix
[[10, 1], [0, 0]]
Accuracy :=> 90.9090909090909

Fold  2
Confusion matrix
[[1, 3], [1, 6]]
Accuracy :=> 63.63636363636363

Fold  3
Confusion matrix
[[1, 2], [1, 7]]
Accuracy :=> 72.72727272727273

Fold  4
Confusion matrix
[[3, 8], [0, 0]]
Accuracy :=> 27.27272727272727

Fold  5
Confusion matrix
[[6, 5], [0, 0]]
Accuracy :=> 54.54545454545454

Fold  6
Confusion matrix
[[7, 4], [0, 0]]
Accuracy :=> 63.63636363636363

Fold  7
Confusion matrix
[[4, 0], [0, 7]]
Accuracy :=> 100.0

Fold  8
Confusion matrix
[[0, 0], [0, 11]]
Accuracy :=> 100.0

Fold  9
Confusion matrix
[[0, 0], [0, 11]]
Accuracy :=> 100.0

Fold  10
Confusion matrix
[[0, 0], [2, 9]]
Accuracy :=> 81.81818181818183

Final confusion matrix
[[32, 23], [4, 51]]

Average accuracy=> 75.45454545454547
